In [1]:
from pytdx.hq import TdxHq_API
from pytdx.params import TDXParams
from datetime import datetime, timedelta
import pandas as pd
import polars as pl
import os
from tqdm import tqdm

HOST = '119.147.212.81'
PORT = 7709

API_TICK_SIZE = 2000

In [2]:
api = TdxHq_API()

In [8]:
df_cn_stks = pl.read_csv('../stk_list/cn_list.csv')
df_cn_stks.head()

df_cn_stks['code'] = df_cn_stks['code'].apply(lambda x: str(x).zfill(6))
df_cn_stks.head()

symbol,name,tdx_api_mkt,code
str,str,i64,str
"""SZ002875""","""安奈儿""",0,"""002875"""
"""SZ002873""","""新天药业""",0,"""002873"""
"""SZ002870""","""香山股份""",0,"""002870"""
"""SZ002869""","""金溢科技""",0,"""002869"""
"""SZ002868""","""绿康生化""",0,"""002868"""


In [9]:
with api.connect(HOST, PORT):
    for i in tqdm(list(range(len(df_cn_stks[0:])))):
        mkt = df_cn_stks['tdx_api_mkt'][i]
        code = df_cn_stks['code'][i]
        
        data = api.get_security_bars(4, mkt, code, 1, 730)
        df_tmp = api.to_df(data)

        df = df_tmp[['open', 'close', 'high', 'low', 'vol', 'amount', 'datetime']]
        df = pl.DataFrame(df)
        df.to_parquet(f'data/{code}.parquet')

100%|██████████| 2024/2024 [05:58<00:00,  5.65it/s]


- 后面干的是把日k chart 和 rvol指标拼接

In [9]:
def get(code):
    df_rvol = pl.read_parquet(f'../cn_data_rvol/data_parquet/{code}.parquet')

    # df_rvol[(len(df_rvol) - 33):]

    df = pl.read_parquet(f'data/{code}.parquet')
    df['rvol_30'] = df_rvol['rvol_30'][(len(df_rvol) - 33):]
    df['datetime'] = df_rvol['time'][(len(df_rvol) - 33):]

    #
    df['volume'] = df['vol'].apply(lambda x: int(x))
    df = df.drop('vol')
    #
    df['symbol'] = [code] * len(df)
    #
    df['freq'] = ['1d'] * len(df)

    # pl.read_parquet('cn_tick_intraday_data.1m.parquet')

    return df
    # df.to_parquet('.1d.parquet')

In [11]:
pl.read_parquet('../cn_data_rvol/data_parquet/002732.parquet')[35:]

date_str,vol,avol_30,rvol_30,freq,time
str,i64,f64,f64,str,datetime
"""20221010""",714,0.0,0.0,"""1d""",2022-10-10 00:00:00
"""20221011""",525,0.0,0.0,"""1d""",2022-10-11 00:00:00
"""20221012""",321,0.0,0.0,"""1d""",2022-10-12 00:00:00
"""20221013""",429,0.0,0.0,"""1d""",2022-10-13 00:00:00
"""20221014""",624,0.0,0.0,"""1d""",2022-10-14 00:00:00


In [11]:
get('002732')

open,close,high,low,rvol_30,datetime,volume,symbol,freq
f64,f64,f64,f64,f64,datetime,i64,str,str
17.66,17.4,17.68,17.37,2.017,2022-11-23 00:00:00,788100,"""002732""","""1d"""
17.4,17.32,17.56,17.18,1.043,2022-11-24 00:00:00,633818,"""002732""","""1d"""
17.38,17.6,17.66,17.17,0.755,2022-11-25 00:00:00,776739,"""002732""","""1d"""
17.57,17.22,17.61,17.0,1.122,2022-11-28 00:00:00,1091821,"""002732""","""1d"""
17.22,17.72,17.87,17.22,1.905,2022-11-29 00:00:00,1180087,"""002732""","""1d"""
17.72,18.0,18.02,17.58,0.934,2022-11-30 00:00:00,1248287,"""002732""","""1d"""
18.5,19.8,19.8,18.06,11.207,2022-12-01 00:00:00,4486600,"""002732""","""1d"""
21.78,21.78,21.78,21.78,22.638,2022-12-02 00:00:00,2803391,"""002732""","""1d"""
23.96,23.96,23.96,23.6,20.187,2022-12-05 00:00:00,21641294,"""002732""","""1d"""


In [10]:
dfs = []

for i in tqdm(list(range(len(df_cn_stks[:100])))):
    mkt = df_cn_stks['tdx_api_mkt'][i]
    code = df_cn_stks['code'][i]

    dftmp = get(code)
    dfs.append(dftmp)

pl.concat(dfs).to_parquet('../cn_data_rvol/visual_output/100stks_rvol.1d.parquet')

100%|██████████| 100/100 [00:00<00:00, 2062.07it/s]
